In [1]:
# -*- coding: utf-8 -*-

#Convert mutliclasses into Single Class

#Resolution: 0.004

#Created on Mon Apr 11 16:42:38 2022

#@author: li.chao.987@s.kyushu-u.ac.jp


from osgeo import gdal
import numpy as np
import os

In [2]:
def createSingleClassTiff (year, aimFolder):
    original_raster = gdal.Open("F:\\17_Article\\01_Data\\01_LandCover\\LandCoverMerge\\LandCover_UMD_" + str(year) + "_500m.tif")
    #print(original_raster)

    raster_rprj = gdal.Warp(aimFolder + "LandCover_UMD_"+ str(year) +"_0004_WGS.tif", original_raster, dstSRS = "EPSG:4326", xRes = 0.004, yRes = 0.004, resampleAlg = "mode")
    print(raster_rprj)
    #raster_rprj = None

    original_raster = None

    band = raster_rprj.GetRasterBand(1)

    multiLista = band.ReadAsArray()

    # reclassification
    singleRasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/"
    driver = gdal.GetDriverByName("GTiff")
    aim = 0
    while aim < 16:
        print(aim)
        originalToSingleLista = multiLista.copy()
        orignialToSingleListaClip = originalToSingleLista[:,37750*2:41750*2].copy()
        #orignialToSingleListaClip = 
        orignialToSingleListaClip[orignialToSingleListaClip == aim] = 100
        orignialToSingleListaClip[orignialToSingleListaClip != 100] = 0
        
        #for j in range(raster_rprj.RasterXSize):
        #    for i in range(raster_rprj.RasterYSize):
        #        if originalToSingleLista[i, j] == aim:
        #            originalToSingleLista[i, j] = 1
        #        else:
        #            originalToSingleLista[i, j] = 0
                    
        # create new file
        name = "year_" + str(year) + "_class_" + str(aim) + ".tif"
        singleRaster = driver.Create(singleRasterLocation + name, 8000, 7500, 1)
        singleRaster.GetRasterBand(1).WriteArray(orignialToSingleListaClip)
        
        proj = raster_rprj.GetProjection()
        georef = (122.0, 0.004, 0.0, 50.0, 0.0, -0.004)
        singleRaster.SetProjection(proj)
        singleRaster.SetGeoTransform(georef)
        singleRaster.FlushCache()
        singleRaster = None
        aim += 1
        
    raster_rprj = None   

    os.remove(aimFolder + "LandCover_UMD_" + str(year) + "_0004_WGS.tif")


In [14]:
aimFolder = "F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS\\"
#os.mkdir("F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS")

year=2001
# to 2020

import multiprocessing

multiprocessing.cpu_count()

12

In [15]:
from joblib import Parallel, delayed
import multiprocessing

Parallel(n_jobs=8)(delayed(createSingleClassTiff)(int(year), aimFolder) for year in np.linspace(2001, 2020, 20))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [16]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from scipy.spatial import cKDTree

# make the raster -180 -60 180 90
nx = 8000                                       # number of cells in the x direction
ny = 7500                                     # number of cells in the y direction
xmin = 122.002                                  # x coordinate of lower, left cell center 
ymin = 30.002                                     # y coordinate of lower, left cell center 
xsize = 0.004                                   # extent of cells in x direction
ysize = 0.004                                   # extent of cells in y direction
epsg = 4326                                     # get projection

def addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg): # Michael Pyrcz, March, 2018                      
  # makes a 2D dataframe with coordinates based on GSLIB specification
  coords = np.zeros([nx*ny, 3])
  ixy = 0
  for iy in range(nx):
    for ix in range(ny):
      coords[ixy,1] = xmin + ix*xsize  
      coords[ixy,2] = ymin + iy*ysize 
      coords[ixy,0] = ixy + 1
      ixy = ixy + 1
      
  coords = pd.DataFrame(coords, columns = ["order", "X", "Y"])
  gdf = gpd.GeoDataFrame(coords, geometry = gpd.points_from_xy(coords.X, coords.Y))
  gdf = gdf.set_crs(epsg = epsg)
  return (gdf)

def ckdnearest(gdA, gdB, column_name):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name=column_name)
        ], 
        axis=1)

    return gdf

coords_distance_raster = addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg)

In [17]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
coords_distance_raster_result = coords_extration.copy()

In [22]:
# https://www.arcgis.com/home/item.html?id=dabc265276a5402a8645834e9a3365f0
# the data from Esri

japanBoundary = gpd.read_file("F:/17_Article/01_Data/00_mesh/JPN_Country_03degree.shp")
japanBoundary['inBuffer'] = 1
japanBoundary = japanBoundary[['inBuffer', 'geometry']]
japanBoundary.head()


,inBuffer,geometry
0,1,"MULTIPOLYGON (((15512009.759 5052211.941, 1551..."


In [24]:
join_coords_distance_raster = coords_distance_raster.sjoin(japanBoundary.to_crs(epsg = 4326), how='left')
join_coords_distance_raster.head()

,order,X,Y,geometry,index_right,inBuffer
0,1.0,122.002,30.002,POINT (122.00200 30.00200),NaN,NaN
1,2.0,122.006,30.002,POINT (122.00600 30.00200),NaN,NaN
2,3.0,122.010,30.002,POINT (122.01000 30.00200),NaN,NaN
3,4.0,122.014,30.002,POINT (122.01400 30.00200),NaN,NaN
4,5.0,122.018,30.002,POINT (122.01800 30.00200),NaN,NaN


In [27]:

filtered_coords_distance_raster = join_coords_distance_raster[join_coords_distance_raster["inBuffer"] == 1]

In [29]:
coords_distance_raster = None
join_coords_distance_raster = None

In [35]:
import gc

gc.collect()

105

In [34]:
del japanBoundary
gc.collect()

NameError: name 'japanBoundary' is not defined

In [32]:
def buildClassAndDistanceDf(year, coords_extration, coords_distance_raster, coords_distance_raster_result):
    type_num = 0
    while type_num < 16:
    
        rasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/year_" + str(year) + "_class_"+ str(type_num) +".tif"
        
        rasterFile = rasterio.open(rasterLocation)
        rasterArray = rasterFile.read(1)
        
        valueArray = []
        for point in coords_extration['geometry']:
            x = point.xy[0][0]
            y = point.xy[1][0]
            row, col = rasterFile.index(x, y)
            valueArray.append(rasterArray[row, col])
            
        valueArray = np.array(valueArray)
        coords_extration['value'] = valueArray
        coords_extration_aim = coords_extration[coords_extration.value == 100]
        coords_extration_aim = coords_extration_aim[['geometry']]
        
        if coords_extration_aim.shape[0] > 0:
            dist = ckdnearest(coords_distance_raster, coords_extration_aim, "dist_class_"+str(type_num))
            dist_column = dist[["dist_class_"+str(type_num)]]
            coords_distance_raster_result["binary_class_" + str(type_num) + "_" + str(year)] = valueArray / 100
            coords_distance_raster_result["dist_class_" + str(type_num) + "_" + str(year)] = dist_column
        else:
            print("class_"+str(type_num)+" is not in japan")
        print("dist_class_"+str(type_num))
        type_num += 1
    return coords_distance_raster_result

In [36]:
from joblib import Parallel, delayed
import multiprocessing

distanceResult = Parallel(n_jobs=6)(delayed(buildClassAndDistanceDf)(int(year), coords_extration, filtered_coords_distance_raster, coords_distance_raster_result) for year in np.linspace(2001, 2020, 20))

In [42]:
distanceResutl_2001 = distanceResult[0]
distanceResutl_2002 = distanceResult[1]
distanceResutl_2003 = distanceResult[2]
distanceResutl_2004 = distanceResult[3]
distanceResutl_2005 = distanceResult[4]
distanceResutl_2006 = distanceResult[5]
distanceResutl_2007 = distanceResult[6]
distanceResutl_2008 = distanceResult[7]
distanceResutl_2009 = distanceResult[8]
distanceResutl_2010 = distanceResult[9]
distanceResutl_2011 = distanceResult[10]
distanceResutl_2012 = distanceResult[11]
distanceResutl_2013 = distanceResult[12]
distanceResutl_2014 = distanceResult[13]
distanceResutl_2015 = distanceResult[14]
distanceResutl_2016 = distanceResult[15]
distanceResutl_2017 = distanceResult[16]
distanceResutl_2018 = distanceResult[17]
distanceResutl_2019 = distanceResult[18]
distanceResutl_2020 = distanceResult[19]

In [47]:
distanceResutl_2001.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2002.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2003.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2004.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2005.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2006.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2007.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2008.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2009.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2010.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2011.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2012.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2013.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2014.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2015.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2016.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2017.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2018.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2019.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']
distanceResutl_2020.columns = ['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2', 'binary_class_3', 'dist_class_3',
       'binary_class_4', 'dist_class_4', 'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9', 'binary_class_10', 'dist_class_10',
       'binary_class_11', 'dist_class_11', 'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15']

In [48]:
distanceResutl_2001['year'] = 2001
distanceResutl_2002['year'] = 2002
distanceResutl_2003['year'] = 2003
distanceResutl_2004['year'] = 2004
distanceResutl_2005['year'] = 2005
distanceResutl_2006['year'] = 2006
distanceResutl_2007['year'] = 2007
distanceResutl_2008['year'] = 2008
distanceResutl_2009['year'] = 2009
distanceResutl_2010['year'] = 2010
distanceResutl_2011['year'] = 2011
distanceResutl_2012['year'] = 2012
distanceResutl_2013['year'] = 2013
distanceResutl_2014['year'] = 2014
distanceResutl_2015['year'] = 2015
distanceResutl_2016['year'] = 2016
distanceResutl_2017['year'] = 2017
distanceResutl_2018['year'] = 2018
distanceResutl_2019['year'] = 2019
distanceResutl_2020['year'] = 2020

In [53]:
distanceResutl_2001.columns

Index(['G04c_001', 'geometry', 'binary_class_0', 'dist_class_0',
       'binary_class_1', 'dist_class_1', 'binary_class_2', 'dist_class_2',
       'binary_class_3', 'dist_class_3', 'binary_class_4', 'dist_class_4',
       'binary_class_5', 'dist_class_5', 'binary_class_6', 'dist_class_6',
       'binary_class_8', 'dist_class_8', 'binary_class_9', 'dist_class_9',
       'binary_class_10', 'dist_class_10', 'binary_class_11', 'dist_class_11',
       'binary_class_12', 'dist_class_12', 'binary_class_13', 'dist_class_13',
       'binary_class_14', 'dist_class_14', 'binary_class_15', 'dist_class_15',
       'year'],
      dtype='object')

In [56]:
distanceResutl_2001 = pd.DataFrame(distanceResutl_2001.drop(columns='geometry'))
distanceResutl_2002 = pd.DataFrame(distanceResutl_2002.drop(columns='geometry'))
distanceResutl_2003 = pd.DataFrame(distanceResutl_2003.drop(columns='geometry'))
distanceResutl_2004 = pd.DataFrame(distanceResutl_2004.drop(columns='geometry'))
distanceResutl_2005 = pd.DataFrame(distanceResutl_2005.drop(columns='geometry'))
distanceResutl_2006 = pd.DataFrame(distanceResutl_2006.drop(columns='geometry'))
distanceResutl_2007 = pd.DataFrame(distanceResutl_2007.drop(columns='geometry'))
distanceResutl_2008 = pd.DataFrame(distanceResutl_2008.drop(columns='geometry'))
distanceResutl_2009 = pd.DataFrame(distanceResutl_2009.drop(columns='geometry'))
distanceResutl_2010 = pd.DataFrame(distanceResutl_2010.drop(columns='geometry'))
distanceResutl_2011 = pd.DataFrame(distanceResutl_2011.drop(columns='geometry'))
distanceResutl_2012 = pd.DataFrame(distanceResutl_2012.drop(columns='geometry'))
distanceResutl_2013 = pd.DataFrame(distanceResutl_2013.drop(columns='geometry'))
distanceResutl_2014 = pd.DataFrame(distanceResutl_2014.drop(columns='geometry'))
distanceResutl_2015 = pd.DataFrame(distanceResutl_2015.drop(columns='geometry'))
distanceResutl_2016 = pd.DataFrame(distanceResutl_2016.drop(columns='geometry'))
distanceResutl_2017 = pd.DataFrame(distanceResutl_2017.drop(columns='geometry'))
distanceResutl_2018 = pd.DataFrame(distanceResutl_2018.drop(columns='geometry'))
distanceResutl_2019 = pd.DataFrame(distanceResutl_2019.drop(columns='geometry'))
distanceResutl_2020 = pd.DataFrame(distanceResutl_2020.drop(columns='geometry'))

In [58]:
longLandCoverDF = pd.concat(
    [distanceResutl_2001, distanceResutl_2002, distanceResutl_2003, distanceResutl_2004, distanceResutl_2005, distanceResutl_2006, distanceResutl_2007, distanceResutl_2008,
    distanceResutl_2009, distanceResutl_2010, distanceResutl_2011, distanceResutl_2012, distanceResutl_2013, distanceResutl_2014, distanceResutl_2015, distanceResutl_2016,
    distanceResutl_2017, distanceResutl_2018, distanceResutl_2019, distanceResutl_2020]
)

In [59]:
longLandCoverDF.head()

,G04c_001,binary_class_0,dist_class_0,binary_class_1,dist_class_1,binary_class_2,dist_class_2,binary_class_3,dist_class_3,binary_class_4,...,dist_class_11,binary_class_12,dist_class_12,binary_class_13,dist_class_13,binary_class_14,dist_class_14,binary_class_15,dist_class_15,year
0,303660001,1.0,0.291539,0.0,1.057513,0.0,0.569970,0.0,10.401981,0.0,...,0.298759,0.0,1.653036,0.0,1.616993,0.0,1.651342,0.0,0.563049,2001
1,303660002,1.0,0.287726,0.0,1.053933,0.0,0.566795,0.0,10.398548,0.0,...,0.294954,0.0,1.650445,0.0,1.616418,0.0,1.648779,0.0,0.559969,2001
2,303660003,1.0,0.283918,0.0,1.050356,0.0,0.563630,0.0,10.395115,0.0,...,0.291154,0.0,1.647860,0.0,1.615852,0.0,1.646221,0.0,0.556901,2001
3,303660004,1.0,0.280115,0.0,1.046783,0.0,0.560477,0.0,10.391683,0.0,...,0.287360,0.0,1.645280,0.0,1.615297,0.0,1.643670,0.0,0.553844,2001
4,303660011,1.0,0.276317,0.0,1.043212,0.0,0.557334,0.0,10.388251,0.0,...,0.283571,0.0,1.642706,0.0,1.614280,0.0,1.641124,0.0,0.550800,2001


In [60]:
del distanceResutl_2001
del distanceResutl_2002
del distanceResutl_2003
del distanceResutl_2004
del distanceResutl_2005
del distanceResutl_2006
del distanceResutl_2007
del distanceResutl_2008
del distanceResutl_2009
del distanceResutl_2010
del distanceResutl_2011
del distanceResutl_2012
del distanceResutl_2013
del distanceResutl_2014
del distanceResutl_2015
del distanceResutl_2016
del distanceResutl_2017
del distanceResutl_2018
del distanceResutl_2019
del distanceResutl_2020

gc.collect()

156

In [68]:
longLandCoverDF = longLandCoverDF.set_index(['G04c_001','year'])

In [69]:
longLandCoverDF.head()

,,binary_class_0,dist_class_0,binary_class_1,dist_class_1,binary_class_2,dist_class_2,binary_class_3,dist_class_3,binary_class_4,dist_class_4,...,binary_class_11,dist_class_11,binary_class_12,dist_class_12,binary_class_13,dist_class_13,binary_class_14,dist_class_14,binary_class_15,dist_class_15
G04c_001,year,,,,,,,,,,,,,,,,,,,,,
303660001,2001,1.0,0.291539,0.0,1.057513,0.0,0.569970,0.0,10.401981,0.0,1.287947,...,0.0,0.298759,0.0,1.653036,0.0,1.616993,0.0,1.651342,0.0,0.563049
303660002,2001,1.0,0.287726,0.0,1.053933,0.0,0.566795,0.0,10.398548,0.0,1.284095,...,0.0,0.294954,0.0,1.650445,0.0,1.616418,0.0,1.648779,0.0,0.559969
303660003,2001,1.0,0.283918,0.0,1.050356,0.0,0.563630,0.0,10.395115,0.0,1.280243,...,0.0,0.291154,0.0,1.647860,0.0,1.615852,0.0,1.646221,0.0,0.556901
303660004,2001,1.0,0.280115,0.0,1.046783,0.0,0.560477,0.0,10.391683,0.0,1.276392,...,0.0,0.287360,0.0,1.645280,0.0,1.615297,0.0,1.643670,0.0,0.553844
303660011,2001,1.0,0.276317,0.0,1.043212,0.0,0.557334,0.0,10.388251,0.0,1.272543,...,0.0,0.283571,0.0,1.642706,0.0,1.614280,0.0,1.641124,0.0,0.550800


In [70]:
longLandCoverDF.to_pickle("F:/17_Article/01_Data/98_20yearPickles/01_LandCoverClassDistance.pkl")